In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf

# Load the heart disease dataset (replace 'data.csv' with your dataset)
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/heart.csv')
data

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,59,1,1,140,221,0,1,164,1,0.0,2,0,2,1
1021,60,1,0,125,258,0,0,141,1,2.8,1,1,3,0
1022,47,1,0,110,275,0,0,118,1,1.0,1,1,2,0
1023,50,0,0,110,254,0,0,159,0,0.0,2,0,2,1


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1025 entries, 0 to 1024
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1025 non-null   int64  
 1   sex       1025 non-null   int64  
 2   cp        1025 non-null   int64  
 3   trestbps  1025 non-null   int64  
 4   chol      1025 non-null   int64  
 5   fbs       1025 non-null   int64  
 6   restecg   1025 non-null   int64  
 7   thalach   1025 non-null   int64  
 8   exang     1025 non-null   int64  
 9   oldpeak   1025 non-null   float64
 10  slope     1025 non-null   int64  
 11  ca        1025 non-null   int64  
 12  thal      1025 non-null   int64  
 13  target    1025 non-null   int64  
dtypes: float64(1), int64(13)
memory usage: 112.2 KB


In [3]:
# Split the dataset into features (X) and labels (y)
X = data.drop('target', axis=1)
y = data['target']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

In [4]:
from sklearn.preprocessing import StandardScaler

# Feature scaling (standardization)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [5]:
y_train.unique()

array([0, 1])

## Neural Network Model

In [6]:
# Define your neural network architecture
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model with an appropriate loss function and optimizer
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                896       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 3009 (11.75 KB)
Trainable params: 3009 (11.75 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# Randomized Hill Climbing

In [7]:
# Define hyperparameters
max_iterations = 100
max_no_improvement = 50
learning_rate = 0.01

best_accuracy = 0.0
best_weights = model.get_weights()
last_improvement = 0

for iteration in range(max_iterations):
    # Make a small random change to the weights
    new_weights = [w + np.random.normal(0, learning_rate, w.shape) for w in best_weights]

    # Set the new weights in the model
    model.set_weights(new_weights)

    # Train the model with the new weights
    history = model.fit(X_train, y_train, epochs=1, validation_data=(X_val, y_val), verbose=1)

    # Get the validation accuracy
    val_accuracy = history.history['val_accuracy'][0]

    # If the new weights result in improved accuracy, keep them
    if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        best_weights = model.get_weights()
    else:
        # If no improvement is observed for a certain number of iterations, stop
        if iteration - last_improvement > max_no_improvement:
            break

    # Print progress
    print(f"Iteration {iteration+1}/{max_iterations}, Validation Accuracy: {val_accuracy}")

# Set the best weights in the model
model.set_weights(best_weights)

# Evaluate the final model on test data
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy}")

24/24 [==============================] - 5s 25ms/step - loss: 0.5911 - accuracy: 0.7249 - val_loss: 0.4836 - val_accuracy: 0.8659
Iteration 1/100, Validation Accuracy: 0.8658536672592163
24/24 [==============================] - 0s 14ms/step - loss: 0.4446 - accuracy: 0.8455 - val_loss: 0.3657 - val_accuracy: 0.8902
Iteration 2/100, Validation Accuracy: 0.8902438879013062
24/24 [==============================] - 0s 15ms/step - loss: 0.3650 - accuracy: 0.8509 - val_loss: 0.2943 - val_accuracy: 0.9146
Iteration 3/100, Validation Accuracy: 0.9146341681480408
24/24 [==============================] - 0s 14ms/step - loss: 0.3283 - accuracy: 0.8537 - val_loss: 0.2649 - val_accuracy: 0.9268
Iteration 4/100, Validation Accuracy: 0.9268292784690857
24/24 [==============================] - 0s 12ms/step - loss: 0.3056 - accuracy: 0.8848 - val_loss: 0.2477 - val_accuracy: 0.9268
Iteration 5/100, Validation Accuracy: 0.9268292784690857
24/24 [==============================] - 0s 11ms/step - loss: 0.3

In [8]:
X_train.shape[1]

13

# Continues Peak Problem on Randomized Hill Climbing

In [9]:
# Define the Continuous Peaks Problem fitness function
def continuous_peaks_fitness(solution, t):
    n = len(solution)
    max_peak = 0
    current_peak = 0

    for i in range(n):
        if solution[i] == 1:
            current_peak += 1
        else:
            max_peak = max(max_peak, current_peak)
            current_peak = 0

    return max(max_peak, current_peak) >= t

# Generate random binary strings for training, validation, and test datasets
np.random.seed(42)  # Set seed for reproducibility
num_samples = 1025  # Adjust the number of samples as needed
string_length = 13  # Adjust the string length as needed
t_threshold = 10  # Adjust the threshold as needed

X_train = np.random.randint(2, size=(num_samples, string_length))
X_val = np.random.randint(2, size=(num_samples, string_length))
X_test = np.random.randint(2, size=(num_samples, string_length))

# Calculate labels based on the fitness function
y_train = np.array([continuous_peaks_fitness(solution, t_threshold) for solution in X_train])
y_val = np.array([continuous_peaks_fitness(solution, t_threshold) for solution in X_val])
y_test = np.array([continuous_peaks_fitness(solution, t_threshold) for solution in X_test])

# Train the neural network on the training data
model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val), verbose=2)

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Epoch 1/10
33/33 - 1s - loss: 0.1513 - accuracy: 0.9610 - val_loss: 0.0223 - val_accuracy: 0.9990 - 836ms/epoch - 25ms/step
Epoch 2/10
33/33 - 0s - loss: 0.0253 - accuracy: 0.9980 - val_loss: 0.0147 - val_accuracy: 0.9990 - 316ms/epoch - 10ms/step
Epoch 3/10
33/33 - 0s - loss: 0.0240 - accuracy: 0.9980 - val_loss: 0.0134 - val_accuracy: 0.9990 - 269ms/epoch - 8ms/step
Epoch 4/10
33/33 - 0s - loss: 0.0233 - accuracy: 0.9980 - val_loss: 0.0131 - val_accuracy: 0.9990 - 189ms/epoch - 6ms/step
Epoch 5/10
33/33 - 0s - loss: 0.0230 - accuracy: 0.9980 - val_loss: 0.0125 - val_accuracy: 0.9990 - 255ms/epoch - 8ms/step
Epoch 6/10
33/33 - 0s - loss: 0.0227 - accuracy: 0.9980 - val_loss: 0.0124 - val_accuracy: 0.9990 - 177ms/epoch - 5ms/step
Epoch 7/10
33/33 - 0s - loss: 0.0224 - accuracy: 0.9980 - val_loss: 0.0122 - val_accuracy: 0.9990 - 175ms/epoch - 5ms/step
Epoch 8/10
33/33 - 0s - loss: 0.0224 - accuracy: 0.9980 - val_loss: 0.0118 - val_accuracy: 0.9990 - 203ms/epoch - 6ms/step
Epoch 9/10
33/